## Code for making the Todai-Asahi dataset
#### Data Management (Spring/Summer 2018) at OSIPP, Osaka U

### R version

### Preamble

#### Check variables in the workspace

In [ ]:
print(ls())

#### Remove the variables (if you want)

In [92]:
rm(list = ls())

#### Import packages

In [93]:
library(data.table) # install.packages("data.table",repos='http://cran.us.r-project.org')
library(plyr)

####  Set the parent directory as a working directory
- Alternatively, you can specify your local directory where you store input data.

In [94]:
#setwd("..")
setwd("C:/Users/shu/Desktop/local_data_todai-asahi") # setwd("path_to_your_directory")

### Import data
- If your data are very big, use `fread()` instead of `read.table()`.

In [95]:
data2009 <- read.table('input/2009UTASP20150910.csv',header=TRUE, sep=',',na.strings=c("",".","NA"),stringsAsFactors=FALSE) 
data2012 <- read.table('input/2012UTASP20150910.csv',header=TRUE, sep=',',na.strings=c("",".","NA"),stringsAsFactors=FALSE) 
data2014 <- read.table('input/2014UTASP20150910.csv',header=TRUE, sep=',',na.strings=c("",".","NA"),stringsAsFactors=FALSE) 
#data2009 <- fread('input/2009UTASP20150910.csv',na.strings=c("",".","NA"))
#data2012 <- fread('input/2012UTASP20150910.csv',na.strings=c("",".","NA")) 
#data2014 <- fread('input/2014UTASP20150910.csv',na.strings=c("",".","NA"))

In [96]:
help(read.table)

### Check contents

#### Show info of all variables

In [ ]:
print(str(data2014))

#### Show a summary table

In [ ]:
print(summary(data2014))

#### Show first five rows

In [ ]:
print(head(data2014,n=5))
# print(data2014) # If you have used fread to import data, the code automatically show first and last five rows

In [ ]:
print(head(data2014['NAME'],n=5)) # only names
# print(data2014[,'NAME'])  # If you have used fread to import data, the code automatically show first and last five rows

#### Check whether the index is unique

In [101]:
print(sum(duplicated(rownames(data2014))))

[1] 0


#### Show unique values and the total number of unique values

In [ ]:
print(unique(data2014[,'NAME']))
print(paste("Unique obs. = ",length(unique(data2014[,'NAME'])))) 
#print(paste("Unique obs. = ",nrow(unique(data2014[,'NAME'])))) # If you use fread

#### Check NA, NaN, and infinity.

In [ ]:
print(apply(is.na(data2014),2,sum)) # total number of NA by column
print(apply(is.nan(as.matrix(data2014)),2,sum)) # total number of NaN by column
print(apply(is.infinite(as.matrix(data2014)),2,sum)) # total number of inf or -inf by column

In [ ]:
q1_1_counts <- count(data2014, 'Q1_1FA') # tabulate Q1_1FA including NaN
print(q1_1_counts)

### Modify data

#### Redefine values for consistency

In [105]:
data2012['RESULT'] <- mapvalues(unlist(data2012['RESULT']),from=c(3,2,1,0),to=c(4,3,2,1))
data2014['RESULT'] <- mapvalues(unlist(data2014['RESULT']),from=c(3,2,1,0),to=c(4,3,2,1))

In [106]:
data2009['PARTY'] = mapvalues(unlist(data2009['PARTY']),from=1:12,to=c('自民党','民主党','公明党','共産党','社民党','国民新党','みんなの党','新党日本','新党大地','改革クラブ','諸派','無所属'))
data2012['PARTY'] = mapvalues(unlist(data2012['PARTY']),from=1:14,to=c('民主党','自民党','未来の党','公明党','日本維新の会','共産党','みんなの党','社民党','新党大地','国民新党','新党日本','新党改革','諸派','無所属'))
data2014['PARTY'] = mapvalues(unlist(data2014['PARTY']),from=1:13,to=c('自民党','民主党','維新の党','公明党','次世代の党','共産党','生活の党','社民党','新党改革','幸福実現党','支持政党なし','諸派','無所属'))

### Make a panel dataset
- If you want to merge 2009, 2012 and 2014 data, what things do you need to consider?
       - Same variable name?
       - Same survey question?
       - Same meaning (e.g. party = 1 always LDP)?
       - Which variable is the identifier (if there is any)?

#### 1. Rename and keep relevant variables, by election year

In [107]:
# keep attributes like name, age, etc.
attr <- c('ID','NAME','RESPONSE','PREFEC','DISTRICT','PRBLOCK','INCUMB','TERM','PARTY','SEX','AGE','RESULT')

data2009_attr <- subset(data2009,select=attr)
data2012_attr <- subset(data2012,select=attr)
data2014_attr <- subset(data2014,select=attr)

In [108]:
# keep consistent survey questions

# rename columns 
setnames(data2009,c(
    'Q4_1','Q4_2','Q4_3',
    'Q5_1','Q5_2','Q5_3','Q5_4','Q5_5','Q5_6',
    'Q8_1','Q8_4',
    'Q9_1','Q9_2','Q9_4','Q9_5','Q9_6','Q9_9','Q9_11','Q9_12','Q9_14','Q9_16','Q9_17','Q9_18',
    'Q10_1','Q10_7','Q10_9',
    'Q11_6','Q11_9','Q11_12','Q11_13'), c(
    'camp1','camp2','camp3',
    'coal_lib','coal_dem','coal_kmei','coal_com','coal_soc','coal_koku',
    'yn_medconst','yn_reps','yn_const','yn_defense','yn_preemp','yn_unsc','yn_nkorea','yn_smgov','yn_pubspend','yn_fiscalpol','yn_tax5yrs','yn_frgnvote','yn_frgnwork','yn_privacy',
    'ab_env','ab_singlemum','ab_trade',
    'fav_masuzoe','fav_ozawa','fav_watanabe','fav_hashimoto'))

setnames(data2012,c(
    'Q1_1','Q1_2','Q1_3',
    'Q2_1','Q2_2','Q2_3',
    'Q4_1','Q4_2','Q4_4','Q4_5','Q4_7','Q4_9',
    'Q5_1','Q5_2','Q5_3','Q5_4','Q5_5','Q5_7','Q5_8','Q5_9','Q5_10','Q5_11','Q5_15','Q5_16','Q5_17','Q5_18','Q5_21','Q5_25',
    'Q6_1','Q6_2','Q6_3','Q6_4','Q6_5',
    'Q10_3','Q10_6','Q10_8','Q10_16',
    'Q11_1','Q11_3','Q11_4'), c(
    'prim1','prim2','prim3',
    'camp1','camp2','camp3',
    'coal_dem','coal_lib','coal_kmei','coal_com','coal_soc','coal_koku',
    'yn_const','yn_defense','yn_preemp','yn_unsc','yn_nkorea','yn_smgov','yn_pubspend','yn_fiscalpol','yn_tax5yrs','yn_tax10pc','yn_privacy','yn_frgnvote','yn_frgnwork','yn_moral','yn_reps','yn_medconst',
    'ab_asiaus','ab_compet','ab_trade','ab_env','ab_singlemum',
    'fav_ozawa','fav_watanabe','fav_hashimoto','fav_masuzoe',
    'media_hp','media_sns','media_tv'))
                                  
setnames(data2014,c(
    'Q1_1','Q1_2','Q1_3',
    'Q2_1','Q2_2','Q2_3',
    'Q6_1','Q6_2','Q6_3','Q6_5','Q6_6','Q6_7','Q6_8','Q6_11','Q6_12','Q6_13','Q6_14',
    'Q7_1','Q7_2','Q7_3','Q7_4','Q7_5',
    'Q14_1','Q14_2','Q14_3'), c(
    'prim1','prim2','prim3',
    'camp1','camp2','camp3',
    'yn_defense','yn_preemp','yn_nkorea','yn_smgov','yn_pubspend','yn_fiscalpol','yn_tax10pc','yn_privacy','yn_frgnvote','yn_frgnwork','yn_moral',
    'ab_asiaus','ab_compet','ab_trade','ab_env','ab_singlemum',
    'media_hp','media_sns','media_tv'))                                  
    
# keep columns
data2009_subset <- subset(data2009,select=grepl("camp|coal_|yn_|ab_|fav_",names(data2009)))
data2012_subset <- subset(data2012,select=grepl("prim|camp|coal_|yn_|ab_|fav_|media_",names(data2012)))
data2014_subset <- subset(data2014,select=grepl("prim|camp|yn_|ab_|media_",names(data2014)))

In [109]:
# append datasets by row
data2009_use <- cbind(data2009_attr,data2009_subset)
data2012_use <- cbind(data2012_attr,data2012_subset)
data2014_use <- cbind(data2014_attr,data2014_subset)

In [110]:
# add election year
data2009_use$ELECYEAR <- '2009'
data2012_use$ELECYEAR <- '2012'
data2014_use$ELECYEAR <- '2014'

#### 2. Append all years

In [113]:
# append datasets by row
data <- rbind.fill(data2009_use,data2012_use,data2014_use)

# change a duplicated name
data[(data['NAME'] == '金子＝恵美') & (data['ELECYEAR'] == '2014') & (data['PARTY'] == '民主党'),'NAME'] <- '金子＝恵美２'

# add index (name and prefecture)
data$uid <- as.numeric(as.factor(data$NAME))

# reorder columns
data = data[,c(
'uid',
'ELECYEAR',
'ID',
'NAME',
'RESPONSE',
'PREFEC',
'DISTRICT',
'PRBLOCK',
'INCUMB',
'TERM',
'PARTY',
'SEX',
'AGE',
'RESULT',
'prim1',
'prim2',
'prim3',
'camp1',
'camp2',
'camp3',
'coal_lib',
'coal_dem',
'coal_kmei',
'coal_com',
'coal_soc',
'coal_koku',
'yn_medconst',
'yn_reps',
'yn_const',
'yn_defense',
'yn_preemp',
'yn_unsc',
'yn_nkorea',
'yn_smgov',
'yn_pubspend',
'yn_fiscalpol',
'yn_tax5yrs',
'yn_frgnvote',
'yn_frgnwork',
'yn_privacy',
'yn_tax10pc',
'yn_moral',
'ab_asiaus',
'ab_compet',    
'ab_env',
'ab_singlemum',
'ab_trade',
'fav_masuzoe',
'fav_ozawa',
'fav_watanabe',
'fav_hashimoto',
'media_hp',
'media_sns',
'media_tv')]

# check whether uid and ELECYEAR are unique and not missing
print(sum((duplicated(data[,c('uid','ELECYEAR')]))))
print(sum(is.na(data[,c('uid','ELECYEAR')])))

[1] 0
[1] 0


#### 3. Save data

In [115]:
fwrite(data,"output/syuuin_2009_2014_R.csv")